下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,I was not really fond of this place. It had a good vibe and I had really good service. \n\nThe food was not the best thing I had ever tasted and the place was very loud.\n\nI will tell you that the Ahi Tuna burger I got was really good but the sweet potato fries were ok.\n\nI live kind of far for this place and I would rather go to delux for a burger or in and out or chuck box or smashburger.\n\nGood for groups of 2-6\n\nRecommendation: If your close by and need a good lunch stop in but not worth any trek out.
1,1 star,"Rude customer service , microwaved food , Owners will tell you to your face there not interested if they dont wanna accommodate your needs that day. \n\nThey keep moving locations probably because they have worn down there welcome in the community everywhere they go .. 3 different locations in 5 years ."
2,5 stars,"There are good bakeries all over town. When it comes to sugar and cream there isn't a lot that hasn't been done but where a business can set itself apart is its service. The girls working this morning were an absolute pleasure. Friendly, fun and personable. My sales receipt says her name was Jordan and she had great customer service. \n\nI haven't had a chance to go through and try my entire box of goodies but I will say the cannoli was one of the best I have ever had. Their cookies are butter style. More of a crisp texture than a chewy but I saw some cake style ones that I may have to try next time.\n\nOverall I will say this is a high quality bakery and that if you are just looking for something small or something elaborate they would be a great place to start."
3,4 stars,"One of my favorite italian restaurants. I'm convinced I can survive on their antipasto salad and cibata alone. I always switch back and forth between two favorites: spaghetti and meatballs or the baked rigatoni. I also really like the chicken marsala. And if you ever have room, do yourself a favor and order the cappuccino pie. So delicious!"
4,1 star,"This place is awful! Upon entry, the cashier tried to cheat me, wanting the dinner price when it was nowhere near the dinner time posted on the wall. She finally let me in at the posted price. The couple who came in after me & sat near my table argued the whole time I was there, because she had allowed the cashier to cheat them & the guy was having a fit about it!\n\nThere wasn't a bit of service available, nor a drop of regular water to drink, as soda water came out of both \""water\"" taps on the soda fountain. Avoid this place at all costs!\n\nIf you're hungry for something right now, go around the other side of the building to King Wah Buffet. It's pretty bad (* *), but it's worlds above this place. Better yet, drive 3 miles west to Jumbo Buffet (* * * *) at 3414 W. Southern, & enjoy a good meal!"
5,1 star,"I would USUALLY give this place 4.5 stars. Me and my fellow roommates usually have nothing but good things to say. We went there about a month and a half ago. I wasn't going to write a review but my experience left such a bad taste in my mouth that none of us have been back since. Which is unfortunate because we went there a lot. ( we went there the night before the bad experience)\n\nHeres how the night went:\n- After just taking our drink orders the waitress had no idea who's drinks went where. Which would not be a big deal if she didn't say \""sorry I've had a longggg day, so I'm like not going to remember anything\"" (I'm thinking \""uhhh thanks, I feel so welcome\"")\n\n-Never asked us if we needed another drink (when ours were obviously empty)\n\n-Since they don't do separate checks, we initialed and color coded each persons orders, and even wrote the total for each initial at the bottom of the check. She then picks up the check and says \""you have to figure out tax\"" (which in all of the times we have been there we have NEVER had to do before) Even so, If we have to figure out tax why wouldn't it just be included in the price. Some of my friends

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/root/miniconda3/envs/yeqing/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"Chicken n' Waffles. There was a man sitting across from me that I almost did not claim as my husband, because of the utter pleasure he was taking in the fried chicken. To quote him \""I am not sure that I can ever eat fried chicken again after this....it was just too good.\"" The hallowed chicken was also married with a savory waffle and grilled green beans. Flava Flave! That was the end of our meal...and to be frank everything else was pretty damn tasty. One more quote from the old ball and chain \""if you can make SALAD tasty, you know what you are doing!\"" \nStarters were cheese curds that are battered and deep fried and so they ooze out and send wonderful cheese into a blissful place....ok don't start me on cheese. Then cr\u00eapes filled with chicken and caramelized onion (oh how I love a savory Crepe). Mac and cheese was so so good....and dates stuffed with chorizo on a bed of arugula. It was the perfect mix of sweet, spicy, and savory all at once.\nThe atmosphere and service also gets two big \""hollas\"" but to be honest, serve me this food on a busy intersection and I would be happy too.\nRocks.","[101, 18770, 183, 112, 160, 9823, 23445, 1116, 119, 1247, 1108, 170, 1299, 2807, 1506, 1121, 1143, 1115, 146, 1593, 1225, 1136, 3548, 1112, 1139, 2252, 117, 1272, 1104, 1103, 15462, 4687, 1119, 1108, 1781, 1107, 1103, 15688, 9323, 119, 1706, 15882, 1140, 165, 107, 146, 1821, 1136, 1612, 1115, 146, 1169, 1518, 3940, 15688, 9323, 1254, 1170, 1142, 119, 119, 119, 119, 1122, 1108, 1198, 1315, 1363, 119, 165, 107, 1109, 2885, 12595, 9323, 1108, 1145, 1597, 1114, 170, 21718, 12198, 1183, 20049, 13327, 1105, 176, 26327, 1181, 2448, 15154, 119, 143, 24330, 143, 9516, 1162, 106, 1337, 1108, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [13]:
from transformers import TrainingArguments, Trainer
model_dir = "models/bert-base-cased-finetune-yelp"
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:
```shell
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:07.0 Off |                  Off |
| N/A   38C    P0             26W /   70W |     571MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A     39285      C   ...t/miniconda3/envs/yeqing/bin/python        568MiB |
+-----------------------------------------------------------------------------------------+
```

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.242100,1.090886,0.526000
2,0.901400,0.960115,0.591000
3,0.638200,0.978361,0.592000


TrainOutput(global_step=189, training_loss=0.9693943861300353, metrics={'train_runtime': 341.7098, 'train_samples_per_second': 8.779, 'train_steps_per_second': 0.553, 'total_flos': 789354427392000.0, 'train_loss': 0.9693943861300353, 'epoch': 3.0})

In [19]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [20]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.6406865119934082,
 'eval_accuracy': 0.23,
 'eval_runtime': 3.268,
 'eval_samples_per_second': 30.6,
 'eval_steps_per_second': 3.978}

## Homework: 使用完整的 YelpReviewFull 数据集训练

In [14]:
from transformers import TrainingArguments, Trainer
train_dataset = tokenized_datasets["train"].shuffle(seed=32)
eval_dataset = tokenized_datasets["test"].shuffle(seed=32)
model_dir = "models/bert-base-cased-finetune-yelp"
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=1,
                                  logging_steps=2000)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

## 资源使用情况
```shell
Every 1.0s: nvidia-smi                             iZ6wearaq5de2lchqv8aowZ: Sat Aug  2 17:24:03 2025

Sat Aug  2 17:24:03 2025
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:07.0 Off |                  Off |
| N/A   68C    P0             68W /   70W |   11817MiB /  16384MiB |     90%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+

+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A     45412      C   ...t/miniconda3/envs/yeqing/bin/python      11814MiB |
+-----------------------------------------------------------------------------------------+
```

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.710200,0.706040,0.691260


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=40625, training_loss=0.7829521807391827, metrics={'train_runtime': 55337.0073, 'train_samples_per_second': 11.746, 'train_steps_per_second': 0.734, 'total_flos': 1.710267926016e+17, 'train_loss': 0.7829521807391827, 'epoch': 1.0})

In [16]:
trainer.evaluate(eval_dataset)

{'eval_loss': 0.7060398459434509,
 'eval_accuracy': 0.69126,
 'eval_runtime': 1461.0422,
 'eval_samples_per_second': 34.222,
 'eval_steps_per_second': 4.278,
 'epoch': 1.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [17]:
trainer.save_model(model_dir)

In [18]:
trainer.save_state()

In [ ]:
# trainer.model.save_pretrained("./")